# Demo: OpenAI Large Language Model - A RAG Sample

In this demo, we show how to use GPT3 model to analyze natural query, use knowledge base to search for more information and answer questions.

In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('CHAT_COMPLETION_NAME')

### Start with a natural question

In [2]:
input = "Which presendential candidate is likely to win in 2024 election? Explain the reasons."

### Step 1: GPT3: What do I need to to answer this question?

In [3]:
system_prompt=f'''Answer the following questions as best you can. You have access to the following tools:

Web Search: Use the web to find information

You need to follow STRICTLY below format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Research]
Action Input: the input to the action

Begin!
'''
user_prompt = f'''
Question: {input}
Thought:'''

In [5]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

I should research the current potential presidential candidates and any relevant information related to the 2024 election to see the projections and reasoning behind their potential chances of winning.
Action: Research
Action Input: "Potential presidential candidates 2024"


### Step 2: Search web for more details 

In [6]:
# Use Azure BING Search API to get the results
import requests
import json
url = "https://api.bing.microsoft.com/v7.0/search"
headers = {"Ocp-Apim-Subscription-Key": os.getenv('BING_SUBSCRIPTION_KEY')}
params = {"q": "2024 presidential election candidates and current political analysis"}
response = requests.get(url, headers=headers, params=params)

In [7]:
context = ''
for result in response.json()['webPages']['value']:
    context += result['snippet'] + '\n'

### Collected information for further processing

In [8]:
print(context)

Francis Suarez The Democrats Incumbent Joe Biden Dean Phillips Marianne Williamson Third Party Robert F. Kennedy Jr. Cornel West Jill Stein The Republicans Back to Top Former President of the...
Who Are the 2024 Presidential Election Candidates? - The New York Times Advertisement Who’s Running for President in 2024? By Martín González Gómez and Maggie Astor Updated Feb. 25,...
Comment We asked each of the 2024 presidential candidates the same questions about abortion, climate, crime and guns, the economy, education, elections, foreign policy and immigration. If...
Who’s running? Who has dropped out? Election news Republicans under scrutiny for supporting conception bill in wake of Alabama IVF ruling New York Democrats reject bipartisan congressional map...
30 Things Joe Biden Did as President You Might Have Missed By POLITICO MAGAZINE | February 02, 2024 05:00 AM 55 Things You Need to Know About Nikki Haley By Michael Kruse and Sydney Gold |...
Ana Marie Cox Three 2024 election myths t

In [9]:
system_prompt=f'''Answer the following questions as best you can. Use the context provided below:
{context}

Begin!
'''
user_prompt = f'''
Question: {input}\
Answer:'''

### Step 3: GPT3: What do I need to do to answer this question? (Again)

In [10]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

As an AI language model, I don't have the ability to predict the future, including the outcome of elections. Additionally, factors such as campaign strategies, evolving political landscapes, voter behavior, economic conditions, and unforeseen events can all play a significant role in determining the election outcome.

Furthermore, the 2024 presidential election is still in the future, and a wide range of potential candidates representing various parties and ideologies are in the mix. It's important to note that making predictions about election outcomes involves complex analysis and assessments that go beyond the scope of this platform.

The eventual winner of the 2024 presidential election will be determined by the voters' choices, and will likely be influenced by a multitude of factors unique to that particular election cycle. It's advisable to stay informed about the candidates, their platforms, and the issues at stake, while also understanding that election outcomes can be unpredic

### GPT3: I now have final answer to the question. 